In [145]:
import csv
from collections import defaultdict
from dotenv import load_dotenv
import os
import requests


In [146]:
load_dotenv()
API_KEY = os.getenv('API_KEY')

In [147]:
import pickle

def pickling(path, data):
    file = open(path,'wb')
    pickle.dump(data,file)

def unpickling(path):
    file = open(path, 'rb')
    b = pickle.load(file)
    return b

In [153]:
def get_movie_id(movie_name, movie_year, requests_session):
    """Gets the movie_id from TMDB using the movie_name and year

    Args:
        movie_name (str): Movie's name
        movie_year (str): Movie's release year

    Returns:
        int: Movie TMDB id
    """

    r = requests_session.get(f'https://api.themoviedb.org/3/search/movie?api_key={API_KEY}&language=en-US&page=1&include_adult=false', params={'query':movie_name, 'year':movie_year})
    movie_id = r.json()['results'][0]['id']
    return movie_id

def get_movie_actors(movie_name, movie_year, requests_session, limit=20):
    """Returns the top 'limit' movie actors

    Args:
        movie_name (str): Movie's name
        movie_year (str): Movie's release year
        limit (int, optional): Actors limit. Defaults to 20.

    Returns:
        str[]: A list of movie actors
    """

    actors = []

    movie_id = get_movie_id(movie_name, movie_year, requests_session)
    r = requests_session.get(f'https://api.themoviedb.org/3/movie/{movie_id}/credits?api_key={API_KEY}&language=en-US&')

    actors_json = r.json()['cast'][:limit]
    for actor in actors_json:
        actors.append(actor['name'])

    return actors

In [149]:
requests_session = requests.session()
actors= get_movie_actors("mother !", "2017", requests_session)
actors

['Jennifer Lawrence',
 'Javier Bardem',
 'Ed Harris',
 'Michelle Pfeiffer',
 'Brian Gleeson',
 'Domhnall Gleeson',
 'Jovan Adepo',
 'Amanda Chiu',
 'Patricia Summersett',
 'Eric Davis',
 'Raphael Grosz-Harvey',
 'Emily Hampshire',
 'Abraham Aronofsky',
 'Luis Oliva',
 'Stephanie Ng Wan',
 'Chris Gartin',
 'Stephen McHattie',
 'Ambrosio De Luca',
 'Gregg Bello',
 'Arthur Holden']

In [155]:
%%time

actors_map = defaultdict(int)
actors_movies_map = defaultdict(list)

requests_session = requests.session()

with open('top_movies_list.csv', newline='') as file:
    reader = csv.reader(file)
    for idx, movie in enumerate(reader):
        print(" ".join(movie), "Done.")

        # Get the movies actors
        movie_actors = get_movie_actors(movie[1], movie[2], requests_session)

        # Loop through the actors and add the movies sale to them
        for actor in movie_actors:
            actors_map[actor] += int(movie[3][1:]) # 1: to remove the $ sign
            actors_movies_map[actor].append(movie[1])
        
        if idx >= 200:
            break

pickling("actors_map.pckl", actors_map)
pickling("actors_movies_map.pckl", actors_movies_map)

1 Avatar 2009 $2881837181 Done.
2 Avengers: Endgame 2019 $2797501328 Done.
3 Titanic 1997 $2201647264 Done.
4 Star Wars: Episode VII - The Force Awakens 2015 $2069521700 Done.
5 Avengers: Infinity War 2018 $2048359754 Done.
6 Spider-Man: No Way Home 2021 $1915999807 Done.
7 Jurassic World 2015 $1671537444 Done.
8 The Lion King 2019 $1663250487 Done.
9 The Avengers 2012 $1518815515 Done.
10 Furious 7 2015 $1515341399 Done.
11 Top Gun: Maverick 2022 $1472292161 Done.
12 Frozen II 2019 $1450026933 Done.
13 Avengers: Age of Ultron 2015 $1402809540 Done.
14 Black Panther 2018 $1347597973 Done.
15 Harry Potter and the Deathly Hallows: Part 2 2011 $1342359942 Done.
16 Star Wars: Episode VIII - The Last Jedi 2017 $1332698830 Done.
17 Jurassic World: Fallen Kingdom 2018 $1310466296 Done.
18 Frozen 2013 $1281508100 Done.
19 Beauty and the Beast 2017 $1273576220 Done.
20 Incredibles 2 2018 $1243089244 Done.
21 The Fate of the Furious 2017 $1236005118 Done.
22 Iron Man 3 2013 $1214811252 Done.


KeyboardInterrupt: 

In [87]:
def print_actors_map(actors_map, actors_movies_map, limit=20):
    sorted_actors_map = dict(sorted(actors_map.items(), key=lambda x: x[1], reverse=True))
    idx = 0
    for actor_name, profit in sorted_actors_map.items():
        movies_list = actors_movies_map[actor_name]
        print(idx, actor_name, profit, movies_list)
        idx += 1
        if idx > limit:
            break

In [151]:
from IPython.display import display, HTML
import pandas as pd
import numpy as np

def pretty_print(df):
    return display(HTML(df.to_html().replace("\\n","<br>")))

def print_actors_map_pd(actors_map, actors_movies_map, limit=20):

    sorted_actors_map = dict(sorted(actors_map.items(), key=lambda x: x[1], reverse=True))

    data = []
    idx = 0
    for actor_name, profit in sorted_actors_map.items():
        movies_list = actors_movies_map[actor_name]
        data.append([actor_name, f"${profit:,}", "\n".join(movies_list)])
        idx += 1
        if idx > limit:
            break

    df = pd.DataFrame(data, columns=["Actor Name", "Profit", "Movies"])
    # df = df.style.set_properties(**{'text-align': 'left'})
    pretty_print(df)

In [152]:
print_actors_map_pd(actors_map, actors_movies_map)

,Actor Name,Profit,Movies
0,Robert Downey Jr.,"$12,769,379,805",Avengers: EndgameAvengers: Infinity WarThe AvengersAvengers: Age of UltronIron Man 3Captain America: Civil WarSpider-Man: HomecomingIron Man 2Iron ManSherlock Holmes: A Game of Shadows
1,Scarlett Johansson,"$11,860,071,780",Avengers: EndgameAvengers: Infinity WarThe AvengersAvengers: Age of UltronCaptain America: Civil WarThe Jungle BookCaptain America: The Winter SoldierSingIron Man 2
2,Samuel L. Jackson,"$11,593,692,817",The AvengersAvengers: Age of UltronIncredibles 2Spider-Man: Far from HomeCaptain MarvelJurassic ParkStar Wars: Episode III - Revenge of the SithCaptain America: The Winter SoldierStar Wars: Episode II - Attack of the ClonesThe IncrediblesIron Man 2Kong: Skull Island
3,Benedict Cumberbatch,"$11,170,645,978",Avengers: EndgameAvengers: Infinity WarSpider-Man: No Way HomeThe Hobbit: The Battle of the Five ArmiesThe Hobbit: The Desolation of SmaugDoctor Strange in the Multiverse of MadnessThor: RagnarokDoctor Strange
4,Chris Hemsworth,"$10,026,479,425",Avengers: EndgameAvengers: Infinity WarThe AvengersAvengers: Age of UltronThor: RagnarokThor: Love and ThunderThor: The Dark World
5,Zoe Saldaña,"$10,019,068,476",AvatarAvengers: EndgameAvengers: Infinity WarGuardians of the Galaxy Vol. 2Guardians of the GalaxyPirates of the Caribbean: The Curse of the Black Pearl
6,Tom Holland,"$9,927,293,305",Avengers: EndgameAvengers: Infinity WarSpider-Man: No Way HomeCaptain America: Civil WarSpider-Man: Far from HomeSpider-Man: Homecoming
7,Andy Serkis,"$9,780,632,807",Star Wars: Episode VII - The Force AwakensBlack PantherStar Wars: Episode VIII - The Last JediThe Lord of the Rings: The Return of the KingThe Lord of the Rings: The Two TowersThe Lord of the Rings: The Fellowship of the RingThe BatmanDawn of the Planet of the ApesKing Kong
8,Chris Evans,"$9,635,245,136",Avengers: EndgameAvengers: Infinity WarThe AvengersAvengers: Age of UltronCaptain America: Civil WarCaptain America: The Winter Soldier
9,Alan Tudyk,"$9,566,628,841",Frozen IIFrozen IIFrozenTransformers: Dark of the MoonRogue One: A Star Wars StoryZootopiaIce Age: Continental DriftBig Hero 6Moana


## Main

In [ ]:

print_actors_map_pd(actors_map, actors_movies_map)